## Generating an interactive climate map in Plotly (& Geopandas)

---

  
- **DATA**:

     - The data for this particular lesson is **synthetic** / **fake!** It was originally scraped from [Berkeley Earth](http://berkeleyearth.lbl.gov/country-list/), but then regenerated based a simple polynomial trend and monthly seasonality modeled from the original data source & injected with random noise. Hence, the data in this repo is somewhat realistic but not authentic.


- **INSTALLATION**:

    - **Make sure you already have plotly and geopandas installed**! (e.g. `pip` or `conda`), e.g.:

        - `pip install plotly`
    
        - `conda install -c conda-forge geopandas --yes`
    

### Read in historical temperature data

In [ ]:
#Read in the data with pandas
import pandas as pd

df =  pd.read_csv('./data/TG_STAID002759.txt',header=14, index_col=1, parse_dates=True)

In [ ]:
from format import format_data_frame
df = format_data_frame(df, set_index_time_step=False)
df

### Group / aggregate the temperature anomaly data by country, year
- For simplicity, we're only interested in yearly averages

In [ ]:
df['country'] = 'Germany'

df['year'] = df.index.year

In [ ]:
df = df.groupby(['country', 'year']).mean()
df


In [ ]:
#save the group result as a dataframe and reset the index so that it becomes easier later to subselect a year
df = df.reset_index()
df

### Read in the geographic data (geometric shapes of all countries in the world) 
- Hint: Use GeoPandas
    - What is a **Shape file (.shp)?**
        - https://en.wikipedia.org/wiki/Shapefile#Shapefile_shape_format_(.shp)
- Publicly available GIS data downloaded from *Natural Earth*: https://www.naturalearthdata.com/downloads/110m-cultural-vectors/

In [ ]:
#Read in the shapefile with geopandas
import geopandas as gpd
gdf = gpd.read_file('./data/ne_110m_admin_0_countries.shp')
# type(gdf['geometry'])
gdf

In [ ]:
#Select the ADMIN and geometry column
gdf = gdf[['ADMIN', 'geometry']]
gdf


### Time for Visualization with Ploty


### Plot data on a map for a single year (we can make it interactive later)
- Import the **"Plotly Express"** module (usually imported as ``px``), which contains high-level functions that can create entire figures at once. Plotly Express is a built-in part of the plotly library, and is the recommended starting point for creating most common figures. 
    - "Plotly Express is a terse, consistent, high-level API for creating figures." This means that it's a lot easier to use (less code) than the alternative option, which is to manually create, manipulate, configure, and render the underlying "Graph Objects" yourself.

In [ ]:
import plotly.express as px

#### Generate a GeoJSON string for a single year.
- The Plotly library (as well as many other JavaScript-based mapping libraries) requires the data to be in GeoJSON format.
    - **HINT**: How can you convert a GeoDataFrame into a GeoJSON? Think of how you might do this in regular pandas.
- Let's use the year 2013 as an example.

In [ ]:
# df['temp_c'] = df['TG'].round(2)
df

In [ ]:
df_2013 = df[df['year']==2013]

In [ ]:


# geojson

#### Convert the GeoJSON string to an actual python dictionary
- Strangely enough, the GeoJSON object that is generated from a GeoDataFrame is represented as a string.
- Some plotting libraries can parse the string directly, while others (e.g. Plotly) require the GeoJSON data in python to be an actual dictionary.
- We can use the built-in ``json`` library to convert this string into a dictionary, so that Plotly can understand it.

In [ ]:
import json
geojson = json.loads(geojson)

In [ ]:
# geojson['features'][0]

### STEP 6: Generate an interactive choropleth map of the data for a single year
- Use the convenient ``px.choropleth()`` method to generate the interactive figure, which you can even see directly in your Jupyter Notebook!

In [ ]:
fig = px.choropleth_mapbox(
    mapbox_style='white-bg',      # see other styles: https://plotly.com/python/mapbox-layers/
    data_frame = df,                  # dataframe that contains all years you can use gdf or df
    geojson=geojson,                   # we can still use the JSON data from 2000, assuming the countries are the same over time
    featureidkey='properties.ADMIN',      # name of JSON key within the "properties" value that contains country names
    locations='country',                    # name of the dataframe column that contains country names
    color='temp_c',                       # name of the dataframe column that contains numerical data you want to display (Like hue)
    center={"lat": 51.1657, "lon": 10.4515},
    zoom=1,
    color_continuous_scale="thermal",    # see other options: https://plotly.com/python/builtin-colorscales/
    range_color=(-3, 3),
    #opacity =0.5
)

fig.show()

### STEP 7: Export the figure to an HTML file, so you can open it in your web browser!
- The reason the figure is interactive (e.g. you can zoom around, it has hover effects, etc.) is because there's a of front-end (i.e. client-side) JavaScript code that Plotly creates for you automatically.
- Use the ``.write_html()`` method to export the file and open it up in a web browser.
    - Bonus: Open the ``.html`` file in a text editor and see if you can understand any of it :)

In [ ]:
fig.write_html("2013_map.html", include_plotlyjs='cdn')

In [ ]:
#bash command to open the file (should use your web browser by default)

#google-chrome 2013_map.html # for linux
# !open 2013_map.html # for mac
#!start 2013_map.html # for windows

---

### STEP 8: Add an interactive slider / animation to toggle between multiple years in a single file.
- For this, we can use the convenient ``px.choropleth_mapbox()`` method that already has some built-in, front-end animation features right out of the box.
- This time, pass the **entire DataFrame** (not just a single year slice) as an argument into the function.
    - **WARNING**: While the HTML script seems to have fairly good performance right out-of-the-box, the resulting HTML file is pretty massive (~50 MB on hard disk), so you might want to delete the file afterwards!

In [ ]:
fig2 = px.choropleth_mapbox(
    mapbox_style='white-bg',      # see other styles: https://plotly.com/python/mapbox-layers/
    data_frame = df,                  # dataframe that contains all years you can use gdf or df
    geojson=geojson,                   # we can still use the JSON data from 2000, assuming the countries are the same over time
    featureidkey='properties.ADMIN',      # name of JSON key within the "properties" value that contains country names
    locations='country',                    # name of the dataframe column that contains country names
    color='temp_c',                       # name of the dataframe column that contains numerical data you want to display
    center={"lat": 51.1657, "lon": 10.4515},
    zoom=2,
    animation_frame='year',             # name of dataframe column that you want to make frames of
    animation_group='country',   
    color_continuous_scale="thermal",    # see other options: https://plotly.com/python/builtin-colorscales/
    range_color=(-3, 3),
    color_continuous_midpoint=0,
    opacity =0.5)

fig2.show()

In [ ]:
fig2.write_html("all_years_interactive.html", include_plotlyjs='cdn')
#this could take up to minute to generate -- file is very LARGE!

In [ ]:
!open all_years_interactive.html

---

---

In [ ]:
fig2.show()

---

### Next Steps / Bonus / Further Exploration

If you got everything working (congratulations!) and enjoyed using a high-level, interactive, JavaScript-based visualization library like Plotly, then you should have a look at [Dash](https://plotly.com/dash/).
- What is **Dash**?
    - As of 2020, Dash is a super popular framework for building entire data visualization applications, not just "static" HTML files.
    - You can think of it as an extension to Plotly. It's actually built on top of 3 libraries:
        - *Plotly* itself (Dash was actually made by the same developers);
        - *Flask* (a web server framework in python); and
        - *React.js* (a powerful JavaScript library for building really nice user interfaces).
    - It's basically a visualization library that can be used to make entire, sophisticated dashboards that are powered by a back-end web server. Here are some [example of machine-learning visualization apps](https://dash-gallery.plotly.host/Portal/?search=[Machine%20Learning]) built using Dash.

### How to get started with Dash?
- It's a python library, so a simple ``pip install dash`` should work fine.
- I'd love to eventually include a simple step-by-step example as part of this tutorial, but that's still a work in progress...

In the meanwhile, here are places to go for code snippets and inspiration:
- [Dash Sample App](https://github.com/plotly/dash-sample-apps/tree/master/apps/dash-opioid-epidemic) *(from the Plotly developers)*. Check out all the other [amazing stuff you can build with Plotly and Dash](https://dash-gallery.plotly.host/Portal/)!
- The [great code snippets / examples from the Plotly website](https://plotly.com/python/maps/)
- The [Capital Bike Dashboard Project](https://github.com/GesaJo/Capital-Bike-Dashboard) by [Gesa Johannsen](https://github.com/GesaJo), fellow Data Science Instructor @ Spiced Academy.
- *Towards Data Science Article* by [Jun Ye](https://github.com/Perishleaf): [Build an Interactive Climate Choropleth Map with Plotly and Dash](https://towardsdatascience.com/build-an-interactive-choropleth-map-with-plotly-and-dash-1de0de00dce0)
    - and the source code: https://github.com/Perishleaf/data-visualisation-scripts/tree/master/dash_project_medium

**Note**:

Keep in mind that these are massive, powerful libraries that by no means are you expected to learn as part of this course! This tutorial / exercise is more meant for you to get a taste of what's possible in terms of interactive web-based visualization in Python. Learning these libraries is, quite frankly, a matter of just following a couple tutorials, trying it out yourself for a project that interests you, and sometimes digging deep into documentation and/or StackOverflow. This could be a worthwhile time investment as part of a final project!